In [ ]:
"""
Тестовое задание для разработчика go

В базе знаний есть информация о кодах ответа нашего API.

Необходимо написать скрипт, который парсит эту табличку и переносит ее в гуглодоку. Предусмотреть, что в будущем необходимо будет синхронизировать данные в гуглодоке, если что-то изменится в базе знаний.

В результате нужно прислать:

ссылку на код на гитхабе;

ссылку на гуглодоку с перенесенной табличкой;

информацию, сколько времени заняло выполнение задания.

Предпочтительный язык для выполнения задания — go. Допустимые языки — python, ruby.

а тут коды API

https://docs.google.com/document/d/1M9pgqRwTLc9bxWYzdSit0355w4QBhCbnsSH7HXnhv-w/edit
"""

In [1]:
"""
Setup to use Google Api
"""


import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request


def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    # print(pickle_file)

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

def convert_to_RFC_datetime(year=1900, month=1, day=1, hour=0, minute=0):
    dt = datetime.datetime(year, month, day, hour, minute, 0).isoformat() + 'Z'
    return dt

In [212]:
"""
Setup to use google docs api
"""

CLIENT_FILE = 'Client_file.json'
API_NAME = 'docs'
API_VESRION = 'v1'
SCOPES_docs = ['https://www.googleapis.com/auth/documents.readonly']
service_docs = Create_Service(CLIENT_FILE,API_NAME,API_VESRION,SCOPES_docs)

Client_file.json-docs-v1-(['https://www.googleapis.com/auth/documents.readonly'],)
['https://www.googleapis.com/auth/documents.readonly']
docs service created successfully


In [ ]:
"""
Setup to use google sheets api
"""

CLIENT_FILE = 'Client_file.json'
API_NAME = 'sheets'
API_VESRION = 'v4'
SCOPES_docs = ['https://www.googleapis.com/auth/spreadsheets']
service_sheets = Create_Service(CLIENT_FILE,API_NAME,API_VESRION,SCOPES_docs)

In [213]:
"""
Input of google doc file id
"""
file_docs = '1M9pgqRwTLc9bxWYzdSit0355w4QBhCbnsSH7HXnhv-w'

In [ ]:
"""
Input of google sheets file id
"""
file_sheets = '13Lxq2phDyPFuyKO9ClaQTN8KAfoBgMPOUlGimo2_40o'

In [180]:
"""
Get all data from google docs table
Source: https://developers.google.com/docs/api/samples/extract-text
Modified to copy bulletpoints
"""

def read_paragraph_element(element):
    """Returns the text in the given ParagraphElement.

        Args:
            element: a ParagraphElement from a Google Doc.
    """
    text_run = element.get('textRun')
    if not text_run:
        return ''
    return text_run.get('content')


def read_strucutural_elements(elements):
    """Recurses through a list of Structural Elements to read a document's text where text may be
        in nested elements.

        Args:
            elements: a list of Structural Elements.
    """
    text = ''
    
    for value in elements:
        if 'paragraph' in value:
            elements = value.get('paragraph').get('elements')
            for elem in elements:
                text += read_paragraph_element(elem)
        elif 'table' in value:
            # The text in table cells are in nested Structural Elements and tables may be
            # nested.
            table = value.get('table')
            for r,row in enumerate(table.get('tableRows')):
                cells = row.get('tableCells')
                cells_f = []
                for c,cell in enumerate(cells):
                    data = read_strucutural_elements(cell.get('content'))
                    cnt = 0
                    for value in cell.get('content'):
                        if 'paragraph' in value:
                            if(value.get('paragraph').get('bullet')):
                                cnt = cnt + 1
                    data = data.replace('\n','\n◉',cnt)
                    print("Row:",r, "Column:",c,"Data:", data)
                    cells_f.append(data)
                rows_f.append(cells_f)
        elif 'tableOfContents' in value:
            # The text in the TOC is also in a Structural Element.
            toc = value.get('tableOfContents')
    return text

In [216]:
"""
Save all data to rows_f
"""
rows_f = []
cells_f = []
document = service_docs.documents().get(documentId=file_docs).execute()
doc_content = document.get('body').get('content')
text = read_strucutural_elements(doc_content)

Row: 0 Column: 0 Data: HTTP-код ответа

Row: 0 Column: 1 Data: Описание

Row: 1 Column: 0 Data: 200

Row: 1 Column: 1 Data: Запрос успешно обработан

Row: 2 Column: 0 Data: 400

Row: 2 Column: 1 Data: Некорректный запрос (невалидный JSON или XML)

Row: 3 Column: 0 Data: 405

Row: 3 Column: 1 Data: Запрос сделан с методом, отличным от GET или POST

Row: 4 Column: 0 Data: 413

Row: 4 Column: 1 Data: Нарушены ограничения:
◉длина параметра query больше 300 символов
◉или количество ограничений в параметре locations больше 100

Row: 5 Column: 0 Data: 500

Row: 5 Column: 1 Data: Произошла внутренняя ошибка сервиса

Row: 6 Column: 0 Data: 503

Row: 6 Column: 1 Data: Нет лицензии на запрошенный сервис



In [226]:
"""
Clear sheet from data and formatting and write new data and formatting to the sheet
"""

worksheet_name = 'Sheet1!'
cell_range_insert = 'A1'

value_range_body = {
    'majorDimension': 'ROWS',
    'values': rows_f
}

#Clear data
service_sheets.spreadsheets().values().clear(
    spreadsheetId=file_sheets,
    range='Sheet1'
).execute()

#clear formatting
body = {
    "requests": [
        {
            "updateCells": {
                "range": {
                    "sheetId": 0
                },
                "fields": "userEnteredFormat"
            }
        }
    ]
}
service_sheets.spreadsheets().batchUpdate(
    spreadsheetId=file_sheets,
    body=body).execute()

#insert new data
service_sheets.spreadsheets().values().update(
    spreadsheetId=file_sheets,
    valueInputOption='USER_ENTERED',
    range=worksheet_name + cell_range_insert,
    body=value_range_body
).execute()


#insert new formatting
requests = []
requests.append({
    'updateSpreadsheetProperties': {
        'properties': {
            'title': document.get('title')
        },
        'fields': 'title'
    }
})
requests.append({
    "updateBorders": {
        "range": {
          "sheetId": 0,
          "startRowIndex": 0,
          "endRowIndex": len(rows_f),
          "startColumnIndex": 0,
          "endColumnIndex": len(rows_f[0])
        },
        "top": {
          "style": "SOLID",
          "width": 1,
        },
        "bottom": {
          "style": "SOLID",
          "width": 1,
        },
        "left": {
          "style": "SOLID",
          "width": 1,
        },
        "right": {
          "style": "SOLID",
          "width": 1,
        },
        "innerHorizontal": {
          "style": "SOLID",
          "width": 1,
        },
        "innerVertical": {
          "style": "SOLID",
          "width": 1,
        }

      }
})
requests.append({
    "autoResizeDimensions":
    {
        "dimensions": {
          "sheetId": 0,
          "dimension": "COLUMNS",
          "startIndex": 0,
          "endIndex": len(rows_f[0])
        }
    }   
})

requests.append({
    "repeatCell": {
        "range": {
          "sheetId": 0,
          "startRowIndex": 0,
          "endRowIndex": 1,
          "startColumnIndex": 0,
          "endColumnIndex": len(rows_f[0])
            
        },

        "cell": {
          "userEnteredFormat": {
            "backgroundColor": {
              "red": 0.0,
              "green": 0.8,
              "blue": 0.8
            },
            "horizontalAlignment" : "CENTER",
          }
        },
        "fields": "userEnteredFormat(backgroundColor,horizontalAlignment)"
    }
    })

requests.append({
    "repeatCell": {
        "range": {
          "sheetId": 0,
          "startRowIndex": 1,
          "endRowIndex": len(rows_f),
          "startColumnIndex": 0,
          "endColumnIndex": len(rows_f[0])
            
        },

        "cell": {
          "userEnteredFormat": {
            "horizontalAlignment" : "LEFT",
            "verticalAlignment": "MIDDLE",
          }
        },
        "fields": "userEnteredFormat(horizontalAlignment,verticalAlignment)"
    }
    })

body = {
    'requests': requests
}
service_sheets.spreadsheets().batchUpdate(
    spreadsheetId=file_sheets,
    body=body).execute()

{'spreadsheetId': '13Lxq2phDyPFuyKO9ClaQTN8KAfoBgMPOUlGimo2_40o',
 'replies': [{}, {}, {}, {}, {}]}